Cours : [DLC] SQL pour Débutant(e)s Jour 6 - Les Outils Avancés 
Lien : https://www.youtube.com/watch?v=mc4zOX1j7_A&list=PLyh35eYRez8cAEBfi1LWN70Cs77bBvCOp&index=6

Github le coin stat : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL
Fichiers Github récupérés : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL/Jour6


In [1]:
import duckdb
from skimpy import skim

In [2]:
# Connection à la librairie duckdb
conn = duckdb.connect()

Différence entre RANK() et DENSE_RANK() :

RANK() :<br>
Attribue des rangs distincts aux lignes ayant des valeurs différentes selon 
les critères spécifiés.
En cas d'ex-aequo, les lignes concernées se voient attribuer le même rang.
Après un ex-aequo, il y a un saut dans l'attribution des rangs suivants. 
Par exemple, si deux lignes sont ex-aequo au rang 2, le rang suivant sera 4.

DENSE_RANK() :<br>
Comme RANK(), attribue le même rang aux lignes ex-aequo.
Cependant, après un ex-aequo, il n'y a pas de saut dans l'attribution des rangs 
suivants. 
Le rang suivant est incrémenté de 1 par rapport au rang précédent.

## Fichier des ventes

In [3]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
sales_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Ventes.csv', header=True)
    """
).df()

In [4]:
# Stats de la DF
skim(sales_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 5     │                                                          │
│ │ Number of columns │ 7      │ │ datetime64  │ 1     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA  ┃ NA %   ┃ mean    ┃ sd     ┃ p0     ┃ p25    ┃ p50   ┃ p75    ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ VenteID            │   0 │      0 │    50.5 │  29.01 │      1 │  25.75 │  50.5 │  75.25 │    100 │ ▇▇▇▇▇▇  │  │
│ │ ClientID           │   0 │      0 │   51.26 │  29.18 │      1 │  27.75 │  53.5 │  77.25 │     99 │ ▆▇▅▇▆▇  │  │
│ │ EmployeID          │   0 │      0 │   50.65 │   29.4 │      2 │     23 │  49.5 │  76.25 │    100 │ ▇▇▇▆▇▇  │  │
│ │ ProduitID          │   0 │      0 │   48.08 │  24.56 │      3 │     29 │    46 │  65.25 │     98 │ ▅▆▇▇▅▃  │  │
│ │ QuantiteVendue     │   0 │      0 │   48.36 │  30.17 │      1 │     22 │  50.5 │     71 │    100 │ ▇▇▃▇▃▇  │  │
│ │ MontantTotal       │   0 │      0 │    8278 │  11580 │  59.98 │   1542 │  2979 │   9249 │  48500 │ ▇▁▁  ▁  │  │
│ └────────────────────┴─────┴────────┴─────────┴────────┴────────┴────────┴───────┴────────┴────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ DateVente        │    0 │       0 │    2020-01-08 12:43:33     │    2023-12-27 15:39:42     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [5]:
# Enregistrement de la DF en BD virtuelle (requête SQL : VIEW)
conn.register('sales_db', sales_df)

## Fichier des employés

In [6]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
employees_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Employes.csv', header=True)
    """
).df()

In [7]:
# Stats de la DF
skim(employees_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ string      │ 5     │                                                          │
│ │ Number of columns │ 6      │ │ int32       │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %   ┃ mean   ┃ sd       ┃ p0   ┃ p25     ┃ p50    ┃ p75     ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ EmployeID        │    0 │      0 │   50.5 │    29.01 │    1 │   25.75 │   50.5 │   75.25 │   100 │ ▇▇▇▇▇▇  │  │
│ └──────────────────┴──────┴────────┴────────┴──────────┴──────┴─────────┴────────┴─────────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                     ┃ NA     ┃ NA %       ┃ words per row              ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Nom                             │      0 │          0 │                          1 │                   100 │  │
│ │ Prenom                          │      0 │          0 │                          1 │                   100 │  │
│ │ Fonction                        │      0 │          0 │                        2.1 │                   207 │  │
│ │ Email                           │      0 │          0 │                          1 │                   100 │  │
│ │ NumeroTelephone                 │      0 │          0 │                          1 │                   100 │  │
│ └─────────────────────────────────┴────────┴────────────┴────────────────────────────┴───────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [8]:
# Enregistrement de la DF en BD virtuelle (requête SQL : VIEW)
conn.register('employees_db', employees_df)

## Requêtes SQL

Donner le classement des produits en fonction de la somme des quantités vendues

In [9]:
conn.execute(
    """
    -- Colonnes à récupérer
    SELECT
        s.ProduitID
        , SUM(s.QuantiteVendue) AS "Quantités vendues"
        -- Fonction de fenêtrage
        , RANK() OVER(
            ORDER BY SUM(s.QuantiteVendue) DESC) AS Classement
    -- BD à récupérer
    FROM sales_db s
    -- TCD
    GROUP BY s.ProduitID
    """
).df()

,ProduitID,Quantités vendues,Classement
0,41,284.0,1
1,37,198.0,2
2,85,188.0,3
3,46,155.0,4
4,58,151.0,5
...,...,...,...
59,86,11.0,60
60,34,8.0,61
61,22,2.0,62
62,36,1.0,63


Donner le classement des employés qui ont réalisé le plus grand chiffre d'affaires

In [13]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        e.Nom
        , e.Prenom
        , ROUND(SUM(s.MontantTotal), 0) AS "Total des ventes"
        -- Fonctions de fenêtrage
        , RANK() OVER(
            ORDER BY ROUND(SUM(s.MontantTotal), 0) DESC) AS Classement
    -- BD à récupérer
    FROM employees_db e
    JOIN sales_db s
    ON e.EmployeID = s.EmployeID
    -- TCD
    GROUP BY
        e.Nom
        , e.Prenom
    """
).df()

,Nom,Prenom,Total des ventes,Classement
0,Edwards,Kirsten,93988.0,1
1,Ramsey,Bryan,45999.0,2
2,Lucero,Lindsey,45499.0,3
3,Thompson,Travis,43999.0,4
4,Wilson,Kathy,34999.0,5
...,...,...,...,...
60,Torres,Ricky,950.0,61
61,King,Elizabeth,840.0,62
62,Peterson,Richard,500.0,63
63,Daniel,Scott,350.0,64
